In [2]:
import pandas as pd
import numpy as np
from scapy.all import *

D:\Program\Anaconda\envs\dip2022\lib\site-packages\scapy\layers\ipsec.py:471: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
D:\Program\Anaconda\envs\dip2022\lib\site-packages\scapy\layers\ipsec.py:485: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


# Chunk Detection

In [8]:

from math import *
import scapy.layers as layers
IP = layers.inet.IP

GET_THRESH = 300 # bytes
DOWN_THRESH = 300  # bytes
VIDEO_CHUNK_GETSIZE = 700 # bytes
AUDIO_CHUNK_GETSIZE = 600 # bytes
NETINFO_NUM = 25

class Chunk():
    def __init__(self, start_time = 0, server_ip='',ttfb = 0, download_time = 0, slack_time = 0, get_size=0, chunk_size = 0, type="", protocol=""):
        self.start_time = start_time
        self.server_ip = server_ip
        self.ttfb = ttfb
        self.download_time = download_time
        self.slack_time = slack_time
        self.get_size = get_size
        self.chunk_size = chunk_size
        self.type = type
        self.protocol = protocol
    def __str__(self):
        return f"{self.start_time} {self.ttfb} {self.download_time} {self.slack_time} {self.get_size} {self.chunk_size} {self.type} {self.protocol}"
    def __repr__(self):
        return f"{self.start_time} {self.ttfb} {self.download_time} {self.slack_time} {self.get_size} {self.chunk_size} {self.type} {self.protocol}"


def isUplink(p):
    # IP = scapy.layers.inet.IP
    return p[IP].src.startswith('192.168.')

def detectAV(c):
    if abs(c.get_size-VIDEO_CHUNK_GETSIZE) > abs(c.get_size-AUDIO_CHUNK_GETSIZE):
        flag=0
    else:
        flag=1
    if c.chunk_size<=80*1024:
        flag=2
    return flag

def chunkDetection(filename):
    a = rdpcap(filename)
    meta_time = float(a[0].time)
    chunk = {}
    chunks = []
    downFlag = {}
    # IP = scapy.layers.inet.IP
    for p in a:
        if p.haslayer(IP):
            ipSrc=p[IP].src
            ipDst=p[IP].dst
            pLen=p[IP].len
            pHdr=p[IP].ihl*4
            ip_time=float(p.time)
            if isUplink(p) and pLen-pHdr > GET_THRESH:
                if ipDst in chunk:
                    chunk[ipDst].slack_time = ip_time - chunk[ipDst].download_time
                    avFlag=detectAV(chunk[ipDst])
                    if avFlag==0:
                        # chunk[ipDst].type='a'
                        chunks.append(chunk[ipDst])
                    elif avFlag==1:
                        # chunk[ipDst].type='v'
                        chunks.append(chunk[ipDst])
                    else:
                        chunk.pop(ipDst)
                        downFlag.pop(ipDst)
                chunk[ipDst] = Chunk(start_time=ip_time, get_size=pLen-pHdr, server_ip=ipDst)
                downFlag[ipDst] = False
            elif not isUplink(p) and pLen > DOWN_THRESH:
                if ipSrc in chunk:
                    if not downFlag[ipSrc]:
                        chunk[ipSrc].ttfb = ip_time
                        downFlag[ipSrc] = True
                    chunk[ipSrc].download_time = ip_time
                    chunk[ipSrc].chunk_size += pLen - pHdr
                    chunk[ipSrc].protocol = p.proto
    
    for c in chunk.values():
        avFlag=detectAV(c)
        if avFlag==0:
            # c.type='a'
            chunks.append(c)
        elif avFlag==1:
            # c.type='v'
            chunks.append(c)
    return meta_time, chunks

filename = 'RequetDataSetNew/A0/PCAP_FILES/baseline_Jan17_exp_31.pcap'
meta_time, chunks=chunkDetection(filename)

In [9]:
print(chunk)

NameError: name 'chunk' is not defined

In [5]:
import time
start = time.time()
filename = 'RequetDataSetNew/A0/PCAP_FILES/baseline_Jan17_exp_31.pcap'
pacpfile = rdpcap(filename)
end = time.time()
print(f"Use Time:{end - start}")
print(f'lenth:{len(pacpfile)}')

Use Time:17.363425970077515
lenth:84540


In [6]:
for i in range(10):
    print(pacpfile[i][IP].len)

253
145
90
379
52
510
90
52
52
1470


In [10]:
GET_THRESH = 300  # bytes
DOWN_THRESH = 300  # bytes
VIDEO_CHUNK_GETSIZE = 700  # bytes
AUDIO_CHUNK_GETSIZE = 600  # bytes
import scapy.layers as layers

def isUplink(p):
    return p[IP].src.startswith('192.168.')
def detectAV(c):
    if abs(c.get_size-VIDEO_CHUNK_GETSIZE) > abs(c.get_size-AUDIO_CHUNK_GETSIZE):
        flag=0
    else:
        flag=1
    if c.chunk_size<=80*1024:
        flag=2
    return flag
chunk = {}
chunks = []
downFlag = {}
#  用show显示包的内容
count=0
# print(pacpfile[0].show())
IP = layers.inet.IP
i = 0
for p in pacpfile:
    if p.haslayer(IP):
        ipSrc = p[IP].src
        ipDst = p[IP].dst
        pLen = p[IP].len
        pHdr = p[IP].ihl*4
        i+=1
        if int(p[IP].ihl) != 5:
            print("shabi")
            print(p[IP].ihl)
        ip_time = float(p.time)
        end_time = ip_time
        # print(p.show())
        if isUplink(p) and pLen-pHdr > GET_THRESH:
            # print(f'ipSrc:{ipSrc}, ipDst:{ipDst}, pLen:{pLen}, pHdr:{pHdr}, ip_time:{ip_time}, end_time:{end_time}')
            if ipDst in chunk:
                chunk[ipDst].slack_time = ip_time - chunk[ipDst].download_time
                avFlag = detectAV(chunk[ipDst])
                if avFlag == 0:
                    # chunk[ipDst].type='a'
                    chunks.append(chunk[ipDst])
                elif avFlag == 1:
                    # chunk[ipDst].type='v'
                    chunks.append(chunk[ipDst])
                else:
                    chunk.pop(ipDst)
                    downFlag.pop(ipDst)
            chunk[ipDst] = Chunk(start_time=ip_time, get_size=pLen-pHdr, server_ip=ipDst)
            downFlag[ipDst] = False
        elif not isUplink(p) and pLen > DOWN_THRESH:
            if ipSrc in chunk:
                if not downFlag[ipSrc]:
                    chunk[ipSrc].ttfb = ip_time
                    downFlag[ipSrc] = True
                chunk[ipSrc].download_time = ip_time
                chunk[ipSrc].chunk_size += pLen - pHdr
                chunk[ipSrc].protocol = p.proto
for c in chunk.values():
    avFlag=detectAV(c)
    if avFlag==0:
        # c.type='a'
        chunks.append(c)
    elif avFlag==1:
        # c.type='v'
        chunks.append(c)
# f"{self.start_time} {self.ttfb} {self.download_time} {self.slack_time} {self.get_size} {self.chunk_size} {self.type} {self.protocol}"
print(i)

shabi
6
shabi
6
shabi
6
shabi
6
84496


In [73]:
p[IP].proto

6

56


In [19]:
columns = ['start_time', 'server_ip', 'type', 'ttfb', 'download_time', 'get_size', 'chunk_size', 'rel_start_time', 'rel_end_time']
df_chunk=pd.DataFrame(columns=columns)


for c in chunks:
    start_time = c.start_time
    server_ip = c.server_ip
    type = c.type
    ttfb = c.ttfb - c.start_time
    download_time = c.download_time - c.start_time
    get_size = c.get_size
    chunk_size = c.chunk_size
    rel_start_time = start_time-meta_time
    rel_end_time = c.download_time - meta_time
    s = pd.Series([start_time, server_ip, type, ttfb, download_time, get_size, chunk_size, rel_start_time, rel_end_time], index=columns)
    df_chunk.loc[len(df_chunk)] = s

av_getsize_gap = df_chunk.get_size.mean()

for i in range(len(df_chunk)):
    df_chunk.loc[i, 'type'] = 'a' if df_chunk.loc[i, 'get_size'] < av_getsize_gap else 'v'

df_chunk.sort_values(by='start_time', inplace=True)
df_chunk.to_csv('results/chunk_detection/A0/baseline_Jan17_exp_30.csv', index=False)

In [17]:
columns = ['start_time', 'server_ip', 'type', 'ttfb', 'download_time', 'get_size', 'chunk_size', 'rel_start_time', 'rel_end_time']
df_chunk=pd.DataFrame(columns=columns)
df_chunk
s = pd.Series([start_time, server_ip, type, ttfb, download_time, get_size, chunk_size, rel_start_time, rel_end_time], index=columns)

In [46]:
df_chunk.shape

(56, 9)

In [20]:
start_time = df_chunk['start_time'].min()
end_time = df_chunk['start_time'].max()

columns=['timestamp']+[str(i) for i in range(1,121)]

df_120features=pd.DataFrame(columns=columns)

for t_msec in range(int(start_time*1000), int(end_time*1000+100), 100):
    s = [t_msec]
    t = t_msec/1000
    for w in range(1, 21):
        period = w*10.0
        if t-period<start_time:
            total_number_of_chunks_v = -1
            avg_chunk_size_v = -1
            download_time_v = -1
            total_number_of_chunks_a = -1
            avg_chunk_size_a = -1
            download_time_a = -1
        else:
            total_number_of_chunks_v = df_chunk[(df_chunk['type']=='v') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)].shape[0]
            avg_chunk_size_v = df_chunk[(df_chunk['type']=='v') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)]['chunk_size'].mean()
            download_time_v = df_chunk[(df_chunk['type']=='v') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)]['download_time'].sum()
            total_number_of_chunks_a = df_chunk[(df_chunk['type']=='a') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)].shape[0]
            avg_chunk_size_a = df_chunk[(df_chunk['type']=='a') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)]['chunk_size'].mean()
            download_time_a = df_chunk[(df_chunk['type']=='a') & (df_chunk['start_time']>t-period) & (df_chunk['start_time']<t)]['download_time'].sum()
        s += [total_number_of_chunks_v, avg_chunk_size_v, download_time_v, total_number_of_chunks_a, avg_chunk_size_a, download_time_a]
    
    df_120features.loc[len(df_120features)] = s

df_120features.to_csv('results/120features/A0/baseline_Jan17_exp_30.csv', index=False)

In [12]:
start_time = df_chunk['start_time'].min()
end_time = df_chunk['start_time'].max()

columns=['timestamp']+[str(i) for i in range(1,121)]

df_120features=pd.DataFrame(columns=columns)

df_audio_chunk=df_chunk[df_chunk.type=='a']
df_video_chunk=df_chunk[df_chunk.type=='v']

audio_chunk_prev = {'chunk_size':[0], 'download_time':[0]}
video_chunk_prev = {'chunk_size':[0], 'download_time':[0]}

for i in range(len(df_audio_chunk)):
    audio_chunk_prev['chunk_size'].append(audio_chunk_prev['chunk_size'][i] + df_audio_chunk.loc[i, 'chunk_size'])
    audio_chunk_prev['download_time'].append(audio_chunk_prev['download_time'][i] + df_audio_chunk.loc[i, 'download_time'])

for i in range(len(df_video_chunk)):
    video_chunk_prev['chunk_size'].append(video_chunk_prev['chunk_size'][i] + df_video_chunk.loc[i, 'chunk_size'])
    video_chunk_prev['download_time'].append(video_chunk_prev['download_time'][i] + df_video_chunk.loc[i, 'download_time'])


for t_msec in range(int(start_time*1000), int(end_time*1000+100), 100):
    s = [t_msec]
    t = t_msec/1000
    for w in range(1, 21):
        l_audio = 0
        r_audio = 0
        l_video = 0
        r_video = 0
        period = w * 10.0
        l_t = t - period
        if l_t<start_time:
            total_number_of_chunks_v = -1
            avg_chunk_size_v = -1
            download_time_v = -1
            total_number_of_chunks_a = -1
            avg_chunk_size_a = -1
            download_time_a = -1
        else:
            while df_audio_chunk.loc[l_audio, 'start_time'] < l_t:
                l_audio += 1
            while df_audio_chunk.loc[r_audio, 'start_time'] <= t:
                r_audio += 1
            while df_video_chunk.loc[l_video, 'start_time'] < l_t:
                l_video += 1
            while df_video_chunk.loc[r_video, 'start_time'] <= t:
                r_video += 1
            total_number_of_chunks_v = r_video - l_video 
            avg_chunk_size_v = video_chunk_prev['chunk_size'][r_video] - video_chunk_prev['chunk_size'][l_video]
            download_time_v = video_chunk_prev['download_time'][r_video] - video_chunk_prev['download_time'][l_video]
            total_number_of_chunks_a = r_audio - l_audio
            avg_chunk_size_a = audio_chunk_prev['chunk_size'][r_audio] - audio_chunk_prev['chunk_size'][l_audio]
            download_time_a = audio_chunk_prev['download_time'][r_audio-1] - audio_chunk_prev['download_time'][l_audio-1]
        s += [total_number_of_chunks_v, avg_chunk_size_v, download_time_v, total_number_of_chunks_a, avg_chunk_size_a, download_time_a]
    
    df_120features.loc[len(df_120features)] = s

df_120features.to_csv('results/120features/A0/baseline_Jan17_exp_30.csv', index=False)

KeyError: 0

In [54]:
import numpy as np
def inet_to_str(inet):
    try:
        return socket.inet_ntop(socket.AF_INET,inet)
    except:
        return False
def isUplink(ip):
    return inet_to_str(ip.src)[:8] == '192.168.' and (ip.len + 14) > GET_THRESH
def isDownLink(ip):
    return inet_to_str(ip.src)[:8] != '192.168.' and (ip.len + 14) > GET_THRESH

In [52]:
import dpkt

filename = 'RequetDataSetNew/A0/PCAP_FILES/baseline_Jan17_exp_31.pcap'
f = open(filename,'rb')
pcap = dpkt.pcap.Reader(f)


times = []
ip_srcs = []
ip_dsts = []
ip_len = []
ip_ihls = []
ip_protos = []
Get_Start_Time = []
Last_Down_Time = []
Down_Time = 0
i = 0
count = 0
for ts,buf in pcap:
    if count==0:
        unix_start = ts
    # ts是时间戳
    times.append(ts)
    eth=dpkt.ethernet.Ethernet(buf)
    #这里也是对没有IP段的包过滤掉
    if eth.type != dpkt.ethernet.ETH_TYPE_IP:
        continue
    ip = eth.data
    ip_src = inet_to_str(ip.src) #
    ip_dst = inet_to_str(ip.dst) #
    ip_srcs.append(ip_src) #  src
    ip_dsts.append(ip_dst) #  dst
    ip_len.append(ip.len)  #  len
    ip_protos.append(ip.p) #  proto:协议号tcp是6
    ip_ihls.append(ip.hl)  #  ihl
    Last_ip = ip
    if isDownLink(ip):
        Down_Time = ts
    if isUplink(ip):
        i+=1
        idx = ip
        Get_Start_Time.append(ts)
        Last_Down_Time.append(Down_Time)
    count +=1
unix_end = ts
print(i)


241


In [60]:
from scapy.all import *
from math import *
import pandas as pd
import dpkt
import numpy as np


GET_THRESH = 300  # bytes
DOWN_THRESH = 300  # bytes
VIDEO_CHUNK_GETSIZE = 700  # bytes
AUDIO_CHUNK_GETSIZE = 600  # bytes

class Chunk():
    def __init__(self, GetTimestamp=0, GetSize=0, DownStart=0, DownEnd=0, DownSize=0, type=0, GetProtocol="", serverIP = ""):
        self.GetTimestamp = GetTimestamp  #发送请求的时间戳
        self.GetSize = GetSize  #Get请求的长度
        self.DownStart = DownStart  #块的第一个下行包的时间戳
        self.DownEnd = DownEnd  #块的最后一个下行包的时间戳
        self.DownSize = DownSize  #块的所有下行包的大小之和
        self.type = type  #类型：视频或音频
        self.GetProtocol = GetProtocol  #协议
        self.serverIP = serverIP
    def getGetTimestamp(self):
        return self.GetTimestamp

    def getDownEnd(self):
        return  self.DownEnd

    def getServerIP(self):
        return self.serverIP

    def getGetSize(self):
        return self.GetSize

    def setType(self, newType):
        self.type = newType

    def __lt__(self, other):
        self_small = True
        A = self.GetTimestamp
        B = other.GetTimestamp
        self_small = A < B
        return self_small

    def detectAV(self):
        # 论文里是根据GetSize, DownSize, GetProtocol来区分视频块，音频块和后台流量的
        flag = 0
        if self.DownSize <= 80*1024:
            flag = 2
        else:
            if abs(self.GetSize-VIDEO_CHUNK_GETSIZE) > abs(self.GetSize-AUDIO_CHUNK_GETSIZE): #判断这个大小更接近视频块还是音频块
                flag = 0
            else:
                flag = 1
        # 基于协议号的筛选
        return flag

    def __str__(self):
        return f"{self.serverIP} {self.GetTimestamp} {self.GetSize} {self.DownStart} {self.DownEnd} {self.DownSize} {self.type} {self.GetProtocol}"

    def __repr__(self):
        return f"{self.serverIP} {self.GetTimestamp} {self.GetSize} {self.DownStart} {self.DownEnd} {self.DownSize} {self.type} {self.GetProtocol}"

def inet_to_str(inet):
    try:
        return socket.inet_ntop(socket.AF_INET, inet)
    except:
        return False

def isUplink(src):
    return src[0:7] == '192.168'

def ChunkDetection(filename):
    #解析pcap文件
    print("Reading pcap file...")

    #读入pcap文件
    f = open(filename, 'rb')
    pcap = dpkt.pcap.Reader(f)

    # meta_time = 0
    i = 0
    #pcap是Reader类，无法切片，目前先这样写
    # for ts, buf in pcap:
    #     i += 1
    #     if i == 1:
    #         meta_time = ts
    #         break
    # end_time = meta_time
    chunks = {}
    chunksValue = []
    downFlag = {}  #用于标识是不是第一个下行报文

    print("Parsing pcap file...")

    times = []
    ip_srcs = []
    ip_dsts = []
    ip_lens = []
    ip_ihls = []
    ip_protos = []
    for ts, buf in pcap:
        # 这里也是对没有IP段的包过滤掉
        eth = dpkt.ethernet.Ethernet(buf)
        if eth.type != dpkt.ethernet.ETH_TYPE_IP:
            continue
        # ts是时间戳
        times.append(ts)
        ip = eth.data
        ip_src = inet_to_str(ip.src)  #
        ip_dst = inet_to_str(ip.dst)  #
        ip_srcs.append(ip_src)  # src
        ip_dsts.append(ip_dst)  # dst
        ip_lens.append(ip.len)  # len
        ip_protos.append(ip.p)  # proto:协议号tcp是6
        ip_ihls.append(ip.hl)  # ihl

    print("Calcilate chunk...")
    sumGetSize = 0
    for n in range(len(ip_srcs)):
        ipSrc = ip_srcs[n]
        ipDst = ip_dsts[n]
        ipSize = ip_lens[n] - ip_ihls[n] * 4
        ipTime = times[n]
        ipProto = ip_protos[n]
        #end_time = ipTime
        if isUplink(ipSrc) and ipSize > GET_THRESH:
            # 上行包，2种情况：（1）已经出现过的，则处理上一个发往这个站点的块，并初始化新块；（2）没有出现过，初始化新块
            if ipDst in chunks:  #意味着以这一ip地址为目的地址的块已经结束，以这一ip地址为目的地址的块马上开始
                #筛选：刚刚结束的块是否是音频或视频块
                avFlag = chunks[ipDst].detectAV()
                if not avFlag == 2:  # 音频块
                    sumGetSize += chunks[ipDst].getGetSize()
                    chunksValue.append(chunks[ipDst])
                else:  # 后台流量
                    chunks.pop(ipDst)  # 抛弃这个块
                    downFlag.pop(ipDst)
            #初始化新块
            chunks[ipDst] = Chunk(GetTimestamp=ipTime, GetSize=ipSize, GetProtocol=ipProto, serverIP = ipDst)
            downFlag[ipDst] = False
        elif not isUplink(ipSrc) and ipSize > DOWN_THRESH:
            if ipSrc in chunks:
            # 下行包，2种情况：（1）是Get请求的第一个下行包，记录时间，更新大小和时间；（2）不是Get请求的第一个下行包，更新大小和时间
                if not downFlag[ipSrc]:
                    chunks[ipSrc].DownStart = ipTime  # 收到第一个下行包的时间
                    downFlag[ipSrc] = True
                chunks[ipSrc].DownEnd = ipTime
                chunks[ipSrc].DownSize += ipSize
                chunks[ipSrc].protocol = ipProto

    for c in chunks.values():
        avFlag = c.detectAV()
        if not avFlag == 2:  # 音频块
            sumGetSize += c.getGetSize()
            chunksValue.append(c)

    # 区分音频块和视频块
    chunkNum = len(chunksValue)
    ave_GetSize = sumGetSize / chunkNum
    for s_chunk in chunksValue:
        if (s_chunk.getGetSize() > ave_GetSize):
            s_chunk.setType(0)
        else:
            s_chunk.setType(1)

    return chunkNum, chunksValue

def getChunkMetrics(chunkNum, chunksValue):
    sortChunks = sorted(chunksValue)
    #顺序：'start_time', 'type', 'ttfb', 'download_time', 'end_time', 'get_size', 'chunk_size'
    output_data = np.zeros((chunkNum, 7))
    i = 0
    for c in chunksValue:
        start_time = c.GetTimestamp
        ttfb = c.DownStart - start_time
        download_time = c.DownEnd - c.DownStart
        end_time = c.DownEnd
        get_size = c.GetSize
        chunk_size = c.DownSize
        type = c.type
        output_data[i] = np.array([start_time, type, ttfb, download_time, end_time, get_size, chunk_size])
        i += 1
    print(i)
    return output_data

filename = 'RequetDataSetNew/A0/PCAP_FILES/baseline_Jan17_exp_31.pcap'
# 1. chunkDetection
chunkNum, chunksValue0 = ChunkDetection(filename)
# 2. chunkMetrics
output = getChunkMetrics(chunkNum, chunksValue0)

Reading pcap file...
Parsing pcap file...
Calcilate chunk...
56


In [61]:
output

array([[1.5162096103863070e+09,
        0.0000000000000000e+00,
        2.4419069290161133e-02, ...,
        1.5162096129504409e+09,
        1.2470000000000000e+03,
        6.2404900000000000e+05],
       [1.5162096110822229e+09,
        1.0000000000000000e+00,
        5.6140422821044922e-03, ...,
        1.5162096145293260e+09,
        3.1000000000000000e+02,
        2.1291900000000000e+06],
       [1.5162096146351621e+09,
        1.0000000000000000e+00,
        3.3547878265380859e-03, ...,
        1.5162096151560409e+09,
        3.1600000000000000e+02,
        1.5870910000000000e+06],
       ...,
       [1.5162100717320600e+09,
        0.0000000000000000e+00,
        1.6271114349365234e-02, ...,
        1.5162100719303150e+09,
        1.2600000000000000e+03,
        1.9892400000000000e+05],
       [1.5162096243067961e+09,
        1.0000000000000000e+00,
        5.7280063629150391e-03, ...,
        1.5162096256747489e+09,
        3.1900000000000000e+02,
        8.9297000000000000e+05]